# Cheese Recommender

## Scraping the data

In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import unicodedata

Recommending cheese based on blah blah blah...  culturecheesemag.com very lenient API see https://culturecheesemag.com/robots.txt

In [2]:
url = 'https://culturecheesemag.com/wp-content/plugins/culture_cheeseLibrary/index.php?letter=a'

In [3]:
# Make request
req = requests.get(url)

In [4]:
# get 403 error, so need to create a user agent header
req

<Response [403]>

In [5]:
# Make request with a user-agent header
req = requests.get(url, headers={'User-agent':'MichaelKnight4714'})

In [6]:
req

<Response [200]>

In [7]:
req.url

'https://culturecheesemag.com/wp-content/plugins/culture_cheeseLibrary/index.php?letter=a'

In [8]:
req.text

'<!doctype html>\n<html lang="en-US">\n  <head>\n    <meta charset="utf-8">\n    <title>culture: the word on cheese</title>\n    <meta name="viewport" content="width=device-width">\n    <meta name="pocket-site-verification" content="f6af8345962084509bc46bbac7cef4" />\n    <meta name="google-site-verification" content="YGOVyZ3aMn6SZIRkYDXYtiNgSpuHInysEJbgwM-yOYo" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0"/>\n    <meta name="msvalidate.01" content="4935C5EF7C01C8653A1D91C794779471" />\n    <link rel="icon" type="image/jpg" href="https://3ner1e34iilsjdn1qohanunu-wpengine.netdna-ssl.com/culture-icon.jpg">\n    <link rel="stylesheet" href="https://3ner1e34iilsjdn1qohanunu-wpengine.netdna-ssl.com/wp-content/themes/culturecheesemag_theme/style.css">\n    <link rel="stylesheet" href="https://3ner1e34iilsjdn1qohanunu-wpengine.netdna-ssl.com/wp-content/themes/culturecheesemag_theme/css/responsive-nav.css">\n    <link href="//netdna.bootstrapcdn.com/font-awesome

In [9]:
if 'json' in req.headers.get('Content-Type'):
    js = req.json()
else:
    print('Response content is not in JSON format.')
    js = 'spam'

Response content is not in JSON format.


In [10]:
req.headers.get('Content-Type')

'text/html; charset=UTF-8'

In [11]:
soup = BeautifulSoup(req.content, 'lxml')

In [12]:
soup

<!DOCTYPE html>
<html lang="en-US">
<head>
<meta charset="utf-8"/>
<title>culture: the word on cheese</title>
<meta content="width=device-width" name="viewport"/>
<meta content="f6af8345962084509bc46bbac7cef4" name="pocket-site-verification"/>
<meta content="YGOVyZ3aMn6SZIRkYDXYtiNgSpuHInysEJbgwM-yOYo" name="google-site-verification"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="4935C5EF7C01C8653A1D91C794779471" name="msvalidate.01"/>
<link href="https://3ner1e34iilsjdn1qohanunu-wpengine.netdna-ssl.com/culture-icon.jpg" rel="icon" type="image/jpg"/>
<link href="https://3ner1e34iilsjdn1qohanunu-wpengine.netdna-ssl.com/wp-content/themes/culturecheesemag_theme/style.css" rel="stylesheet"/>
<link href="https://3ner1e34iilsjdn1qohanunu-wpengine.netdna-ssl.com/wp-content/themes/culturecheesemag_theme/css/responsive-nav.css" rel="stylesheet"/>
<link href="//netdna.bootstrapcdn.com/font-awesome/4.1.0/css/font-awesome.min.css" rel="stylesheet"/>
<link 

In [16]:
cheese_pages = []
for i in range(len(soup.find_all('li', {'class':'cheese'}))):
    print(soup.find_all('li', {'class':'cheese'})[i].find('a')['href'])
    cheese_pages.append(soup.find_all('li', {'class':'cheese'})[i].find('a')['href'])

/cheese-library/Abbaye-de-Belloc
/cheese-library/Abbaye-de-Citeaux
/cheese-library/Abbaye-de-Tamie
/cheese-library/Abbaye-du-Mont-des-Cats
/cheese-library/Abondance
/cheese-library/Acapella
/cheese-library/Adams-Reserve-Cheddar
/cheese-library/Adantino-with-Tarragon
/cheese-library/Aged-Cheddar-Grafton
/cheese-library/Cheddar-Aged-Kerrygold
/cheese-library/Airco
/cheese-library/Almnas-Tegel


In [17]:
cheese_pages

['/cheese-library/Abbaye-de-Belloc',
 '/cheese-library/Abbaye-de-Citeaux',
 '/cheese-library/Abbaye-de-Tamie',
 '/cheese-library/Abbaye-du-Mont-des-Cats',
 '/cheese-library/Abondance',
 '/cheese-library/Acapella',
 '/cheese-library/Adams-Reserve-Cheddar',
 '/cheese-library/Adantino-with-Tarragon',
 '/cheese-library/Aged-Cheddar-Grafton',
 '/cheese-library/Cheddar-Aged-Kerrygold',
 '/cheese-library/Airco',
 '/cheese-library/Almnas-Tegel']

In [18]:
[str(x) for x in list(range(1, 18))]

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17']

In [19]:
## based off of Riley Dallas' code, provided in the youtube Project 3 info session video from May 2018
## https://www.youtube.com/watch?v=5Y3ZE26Ciuk
## ALSO based on Wesley Bosse and Douglas Strodtman's 'CART - Mini-demo using reddit data' DC-Flex lesson 

# make a function that scrapes a url for data.  make sure to use a beautifulsoup for url
def get_chz_pages(letter, pages):
    
    #set initial conditions
    names = []  #initiate a list to contain all of the posts 
    
    
    ### by switching between different lists within the url, its possible to scrape more data in one day
    url = f'https://culturecheesemag.com/wp-content/plugins/culture_cheeseLibrary/index.php?letter={letter}&page={pages}'


    # Perform a get requests on cheese.com
    res = requests.get(url, headers={'User-agent':'MichaelKnight4714'}) 
    # check to make sure not getting error before doing main quest of the code
    # 429 is error - 200 is no errors
    if res.status_code == 200:  

        
        
        soup = BeautifulSoup(res.content, 'lxml')
        
        for i in range(len(soup.find_all('li', {'class':'cheese'}))):
            
            names.append(soup.find_all('li', {'class':'cheese'})[i].find('a')['href'])
        
        
        

    #if code getting an error, do a print message notification and break the for loop    
    else:
        print('ERROR')
        print(res.status_code)
        #break

    time.sleep(1) #sleep for 1 second in between for loops so as not to appear to be DDoS attack 

    
    return names

In [20]:
webpage_names = []

# go through every letter in the alphabet as the cheeses are listed alphabetically on cheese.com
alph = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 
            'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# for every letter in the alphabet, there are at most 17 pages of cheeses
# note that when there are less then 3 pages for a letter, the function will read in
# the cheeses from the last existant page, which will result in scraping duplicates
#page = ['1', '2', '3']

page = [str(x) for x in list(range(1, 18))]

# loop through every letter in the alphabet
for let in alph:
    # loop through every page for each letter
    for p in page:
        webpage_names += get_chz_pages(let, p)

# drop all duplicates by changing the list to a set (whose objects are distinct) and then back to a list
# note that .sort() is neccesary because changing to a set takes it outt of alphabetical order
webpage_names = list(set(webpage_names))

# note that .sort() is neccesary because changing to a set takes it outt of alphabetical order
webpage_names.sort()

# note that .sort() will sort uppercase before lowercase, so need to use sorted(), lambda, and .casefold()
# https://stackoverflow.com/questions/10269701/case-insensitive-list-sorting-without-lowercasing-the-result
webpage_names = sorted(webpage_names, key=lambda s: s.casefold())

In [21]:
# note that we want this list to fix to our dataframe later on
webpage_names

['/cheese-library/Abbaye-de-Belloc',
 '/cheese-library/Abbaye-de-Citeaux',
 '/cheese-library/Abbaye-de-Tamie',
 '/cheese-library/Abbaye-du-Mont-des-Cats',
 '/cheese-library/Abondance',
 '/cheese-library/Acapella',
 '/cheese-library/Adams-Reserve-Cheddar',
 '/cheese-library/Adantino-with-Tarragon',
 '/cheese-library/Aged-Cheddar-Grafton',
 '/cheese-library/Airco',
 '/cheese-library/Almnas-Tegel',
 '/cheese-library/Alpha-Tolman',
 '/cheese-library/Alpine-Gold',
 '/cheese-library/Alpine-Style-Kennebec',
 '/cheese-library/Alter-Schweizer',
 '/cheese-library/Amablu-St-Petes-Select',
 '/cheese-library/Ameribella',
 '/cheese-library/American-Grana',
 '/cheese-library/Anabasque',
 '/cheese-library/Ancho-Goat-Caciotta',
 '/cheese-library/Angelas-Pillow',
 '/cheese-library/Annelies',
 '/cheese-library/Anthill',
 '/cheese-library/Appalachian',
 '/cheese-library/Appenzeller',
 '/cheese-library/Apple-Smoked-Cheddar',
 '/cheese-library/Appletree-Smoked-Cheddar',
 '/cheese-library/Ardrahan',
 '/chees

In [24]:
len(webpage_names)

1244

In [25]:
#for time sake i will save this list of names into a dataframe and save to a csv
web_names = pd.DataFrame(webpage_names)

# save our cheese  dataframe to the file big_cheese.csv
#web_names.to_csv('./data/website_names.csv', index=False)

In [22]:
#read cheese_posts.csv into 'train'
web_names2 = pd.read_csv('./data/website_names.csv')

In [26]:
web_names2.head()

,Unnamed: 0,0
0,0,/cheese-library/Abbaye-de-Belloc
1,1,/cheese-library/Abbaye-de-Citeaux
2,2,/cheese-library/Abbaye-de-Tamie
3,3,/cheese-library/Abbaye-du-Mont-des-Cats
4,4,/cheese-library/Abondance


In [27]:
web_names2.drop(columns='Unnamed: 0', inplace=True)

In [28]:
webpage_names = list(web_names2['0'])

In [29]:
webpage_names[1243]

'/cheese-library/Ziege-Zacke-Blue'

In [30]:
#url2 = 'https://culturecheesemag.com/cheese-library/Raclette'


url2 = 'https://culturecheesemag.com' + webpage_names[1]

# Make request
req = requests.get(url2, headers={'User-agent':'MichaelKnight4714'})

soup = BeautifulSoup(req.content, 'lxml')

#div = soup.find('div', {'class':'col-sm-12 col-md-6'})
section = soup.find('section', {'class':'main-column col-main'})
#para = soup.find_all('p')

In [31]:
section.find('h2').text
#ul.find_all('p')#[0]

'Abbaye de Citeaux'

In [81]:
section.find_all('dt')

[<dt>Producer</dt>,
 <dt>Country</dt>,
 <dt>Region</dt>,
 <dt>Size</dt>,
 <dt>Weight</dt>,
 <dt>Website</dt>,
 <dt>Milk</dt>,
 <dt>Treatment</dt>,
 <dt>Classification</dt>,
 <dt>Rennet</dt>,
 <dt>Rind</dt>,
 <dt>Style</dt>]

In [83]:
section.find_all('dd')[11].text # this leaves no spaces between entries

'Soft-Ripened (Brie-like)Washed-Rind'

In [71]:
list(section.find_all('dd')[11].stripped_strings)#[1]#[11].text#.find_all('br')#.text

['Soft-Ripened (Brie-like)', 'Washed-Rind']

In [73]:
", ".join(str(x) for x in list(section.find_all('dd')[11].stripped_strings))

'Soft-Ripened (Brie-like), Washed-Rind'

In [29]:
section.find_all('p')

[<p>Located near Losne in Burgundy, France, Abbaye de Citaux is home to one of the most traditional and classic monastic washed-rind cheeses of Europe. The monks at the Abbaye de Cîteaux make cheese of the same name using the milk of their own herd of red and white Montbéliarde cows. The cows graze in the flat meadows surrounding the abbey for at least six months of the year.
 </p>,
 <p> </p>,
 <p>Although there is little left of the great Medieval abbey as it was destroyed shortly after the French Revolution, it remains the spiritual center of the Cistercian Order and is open to the public for guided tours. The Abbey has a Benedictine foundation dating back to 1098 when the monks from Cluny settled in Cîteaux, wishing to follow a more disciplined way of life. (It's the Abbey at Citeaux that gave its name to the reformed order of "Cistercians.") The monks originally focused on winemaking, cultivating vines on the Côte d'Or and also building the Château du Clos de Vougeot. During the 19

In [77]:
posts = []
cheese = {}

cheese['Name'] = section.find('h2').text


for i in range(len(section.find_all('dt'))):
    cheese[section.find_all('dt')[i].text] = section.find_all('dd')[i].text
    
    
descript = ''    
for k in range(len(section.find_all('p'))):
    
    #descript.append(section.find_all('p')[k].text)
    descript += section.find_all('p')[k].text
    
cheese['Description'] = descript.replace('\r', '').replace('\xa0', '').replace('\'', '`')

posts.append(cheese)


In [84]:
posts = []
cheese = {}

cheese['Name'] = section.find('h2').text


for i in range(len(section.find_all('dt'))):
    cheese[section.find_all('dt')[i].text] = ", ".join(str(x) for x in list(section.find_all('dd')[i]
                                                                            .stripped_strings))
    
    
descript = ''    
for k in range(len(section.find_all('p'))):
    
    #descript.append(section.find_all('p')[k].text)
    descript += section.find_all('p')[k].text
    
cheese['Description'] = descript.replace('\r', '').replace('\xa0', '').replace('\'', '`')

posts.append(cheese)


In [85]:
cheese

{'Name': 'Abbaye de Citeaux',
 'Producer': 'Frère Joel & Frère Frédéric',
 'Country': 'France',
 'Region': 'Burgundy',
 'Size': '8 in. diameter.  2 in. high',
 'Weight': '2 lbs',
 'Website': '',
 'Milk': 'Cow',
 'Treatment': 'Raw',
 'Classification': 'Semi Soft',
 'Rennet': 'Animal',
 'Rind': 'Washed, Mold Ripened',
 'Style': 'Soft-Ripened (Brie-like), Washed-Rind',
 'Description': 'Located near Losne in Burgundy, France, Abbaye de Citaux is home to one of the most traditional and classic monastic washed-rind cheeses of Europe. The monks at the Abbaye de Cîteaux make cheese of the same name using the milk of their own herd of red and white Montbéliarde cows. The cows graze in the flat meadows surrounding the abbey for at least six months of the year. Although there is little left of the great Medieval abbeyas it was destroyed shortly after the French Revolution, it remains the spiritual center of the Cistercian Order and is open to the public for guided tours. The Abbey has a Benedicti

In [86]:
cheese['Milk']

'Cow'

In [87]:
cheese['Description']#.replace('\r', '').replace('\xa0', '').replace('\\', '')

'Located near Losne in Burgundy, France, Abbaye de Citaux is home to one of the most traditional and classic monastic washed-rind cheeses of Europe. The monks at the Abbaye de Cîteaux make cheese of the same name using the milk of their own herd of red and white Montbéliarde cows. The cows graze in the flat meadows surrounding the abbey for at least six months of the year. Although there is little left of the great Medieval abbeyas it was destroyed shortly after the French Revolution, it remains the spiritual center of the Cistercian Order and is open to the public for guided tours. The Abbey has a Benedictine foundation dating back to 1098 when the monks from Cluny settled in Cîteaux, wishing to follow a more disciplined way of life. (It`s the Abbey at Citeaux that gave its name to the reformed order of "Cistercians.") The monks originally focused on winemaking, cultivating vines on the Côte d`Or and also building the Château du Clos de Vougeot. During the 19th century the Abbey was d

In [88]:
pd.DataFrame([cheese])

,Name,Producer,Country,Region,Size,Weight,Website,Milk,Treatment,Classification,Rennet,Rind,Style,Description
0,Abbaye de Citeaux,Frère Joel & Frère Frédéric,France,Burgundy,8 in. diameter. 2 in. high,2 lbs,,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind","Located near Losne in Burgundy, France, Abbaye..."


In [89]:
## based off of Riley Dallas' code, provided in the youtube Project 3 info session video from May 2018
## https://www.youtube.com/watch?v=5Y3ZE26Ciuk
## ALSO based on Wesley Bosse and Douglas Strodtman's 'CART - Mini-demo using reddit data' DC-Flex lesson 

# make a function that scrapes a url for data.  make sure to use a beautifulsoup for url
def get_chz_posts(page_name):
    
    #set initial conditions
    posts = []  #initiate a list to contain all of the posts 
    
    ### by switching between different lists within the url, its possible to scrape more data in one day
    url = f'https://culturecheesemag.com{page_name}'     


    # Perform a get requests on cheese.com
    res = requests.get(url, headers={'User-agent':'MichaelKnight4714'}) 
    # check to make sure not getting error before doing main quest of the code
    # 429 is error - 200 is no errors
    if res.status_code == 200:  

        
        
        soup = BeautifulSoup(res.content, 'lxml')
        
        section = soup.find('section', {'class':'main-column col-main'})
        
        
        cheese = {}

        cheese['Name'] = section.find('h2').text


        for i in range(len(section.find_all('dt'))):
            cheese[section.find_all('dt')[i].text] = section.find_all('dd')[i].text


        descript = ''    
        for k in range(len(section.find_all('p'))):

            #descript.append(section.find_all('p')[k].text)
            descript += section.find_all('p')[k].text

        cheese['Description'] = descript.replace('\r', '').replace('\xa0', '').replace('\'', '`')

        posts.append(cheese)

    #if code getting an error, do a print message notification and break the for loop    
    else:
        print('ERROR')
        print(res.status_code)
        print(url)
        #break

    time.sleep(1) #sleep for 1 second in between for loops so as not to appear to be DDoS attack 
    
    return cheese

    #return posts

In [90]:
## based off of Riley Dallas' code, provided in the youtube Project 3 info session video from May 2018
## https://www.youtube.com/watch?v=5Y3ZE26Ciuk
## ALSO based on Wesley Bosse and Douglas Strodtman's 'CART - Mini-demo using reddit data' DC-Flex lesson 

# make a function that scrapes a url for data.  make sure to use a beautifulsoup for url
def get_chz_posts(page_name):
    
    #set initial conditions
    posts = []  #initiate a list to contain all of the posts 
    
    ### by switching between different lists within the url, its possible to scrape more data in one day
    url = f'https://culturecheesemag.com{page_name}'     


    # Perform a get requests on cheese.com
    res = requests.get(url, headers={'User-agent':'MichaelKnight4714'}) 
    # check to make sure not getting error before doing main quest of the code
    # 429 is error - 200 is no errors
    if res.status_code == 200:  

        
        
        soup = BeautifulSoup(res.content, 'lxml')
        
        section = soup.find('section', {'class':'main-column col-main'})
        
        
        cheese = {}

        cheese['Name'] = section.find('h2').text


        for i in range(len(section.find_all('dt'))):
            cheese[section.find_all('dt')[i].text] = ", ".join(str(x) for x in list(section.find_all('dd')[i]
                                                                            .stripped_strings))


        descript = ''    
        for k in range(len(section.find_all('p'))):

            #descript.append(section.find_all('p')[k].text)
            descript += section.find_all('p')[k].text

        cheese['Description'] = descript.replace('\r', '').replace('\xa0', '').replace('\'', '`')

        posts.append(cheese)

    #if code getting an error, do a print message notification and break the for loop    
    else:
        print('ERROR')
        print(res.status_code)
        print(url)
        #break

    time.sleep(1) #sleep for 1 second in between for loops so as not to appear to be DDoS attack 
    
    return cheese

    #return posts

In [91]:
for n in webpage_names[0:4]:
    print(n)

/cheese-library/Abbaye-de-Belloc
/cheese-library/Abbaye-de-Citeaux
/cheese-library/Abbaye-de-Tamie
/cheese-library/Abbaye-du-Mont-des-Cats


In [92]:
get_chz_posts('/cheese-library/Abbaye-de-Belloc')

{'Name': 'Abbaye de Belloc',
 'Producer': 'Abbaye de Notre-Dame de Belloc',
 'Country': 'France',
 'Region': 'Aquitaine',
 'Size': '8-9 in. diameter, 3-4 in. high',
 'Weight': '8-10 lbs',
 'Website': '',
 'Milk': 'Sheep',
 'Treatment': 'Raw',
 'Classification': 'Firm',
 'Rennet': 'Animal',
 'Rind': 'Natural, Paprika',
 'Style': 'Alpine-style, Cheddar-like',
 'Flavor': 'Flavor added to rind',
 'Description': 'Abbaye de Belloc is a French cheese that is made by Benedictine monks at the Abbaye de Notre-Dame de Belloc in the Western Pyrenees. The monks developed this sheep’s milk cheese in the 1960s, though its recipe is based on another local classic, Ossau Iraty.Abbaye de Belloc is produced pasteurized sheep`s milk sourced from red-nosed Manech ewes. Farmhouse sheep’s milk cheese production in this mountainous region is resolutely seasonal, with cheesemaking taking place mainly between December and the end of July.After production, wheels of Abbaye de Belloc are aged for four to ten mont

In [93]:
posts = []
#for n in webpage_names[0:3]:
for n in webpage_names:
    posts.append(get_chz_posts(n))
    #posts += get_chz_posts(n)
chz_df = pd.DataFrame(posts)

In [94]:
chz_df

,Name,Producer,Country,Region,Size,Weight,Website,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,"8-9 in. diameter, 3-4 in. high",8-10 lbs,,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...
1,Abbaye de Citeaux,Frère Joel & Frère Frédéric,France,Burgundy,8 in. diameter. 2 in. high,2 lbs,,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",NaN,"Located near Losne in Burgundy, France, Abbaye..."
2,Abbaye de Tamie,Monks of Tamie,France,Savoie,"8 in. diameter, 2-3 in. high",3 lbs,,Cow,Raw,Semi Soft,Animal,Washed,Washed-Rind,NaN,"Made from raw cow`s milk, Abbaye de Tamie is p..."
3,Abbaye du Mont des Cats,Abbaye du Mont des Cats,France,Nord-Pas-de-Calais,"8 in. diameter, 4 in. high",4 lbs,,Cow,NaN,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",NaN,Abbaye du Mont des Cats is a cow`s milk cheese...
4,Abondance,Various,France,Haut Savoie,"2 ¾ in. tall, 15 ¾ in. diameter",15 - 26 lbs,www.fromageabondance.fr/en/,Cow,Raw,Firm,Animal,Natural,Alpine-style,NaN,Named after a valley in the French Alps where ...
5,Acapella,Andante Dairy,United States,California,Varies,Varies,www.andantedairy.com,Goat,Pasteurised,Soft,Microbial,"Mold Ripened, Ash coated",Soft-Ripened (Brie-like),NaN,"Established in 1999, Andante Dairy is owned an..."
6,Adams Reserve Cheddar,Great Lakes Cheese,United States,New York,NaN,40 lbs,www.adamsreserve.com,Cow,Pasteurised,Hard,Vegetable,Plasticoat,Cheddar-like,NaN,Adams Reserve is produced by Great Lakes Chees...
7,Adantino with Tarragon,Andante Dairy,United States,California,"3.5 in. long, 1.5 in. wide, 0.75 in. high",5 oz,www.andantedairy.com,Goat,Pasteurised,Soft,Microbial,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to paste,"Established in 1999, Andante Dairy is owned an..."
8,Aged Cheddar (Grafton),Grafton Village Cheese,United States,Vermont,Varies,8 oz; 1 lb; 5 lbs; 10 lbs,http://www.graftonvillagecheese.com/,Cow,Raw,Firm,Microbial,"None, Plastic",Cheddar-like,NaN,"Located in Vermont, the Grafton Cooperative Ch..."
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"8 ins diameter, 4 ins high",10 lbs,www.carrvalleycheese.com,"Goat, Sheep, Cow",Pasteurised,Semi Soft,Vegetable,Smoked,"Alpine-style, Cheddar-like",Smoked,Located in the rolling hills of central Wiscon...


In [75]:
# save our cheese  dataframe to the file big_cheese.csv
#chz_df.to_csv('./data/culture_cheese_mag.csv', index=False)

In [95]:
# save our cheese  dataframe to the file big_cheese.csv
#chz_df.to_csv('./data/culture_cheese_mag_fixed.csv', index=False)